# 0. Sequential run 

## Config

In [2]:
input = "llama_3.bin"
steps = "500"


## Generate measurements

In [3]:
import subprocess,os
import datetime
from pathlib import Path

commit_hash = subprocess.check_output(["git", "rev-parse", "HEAD"]).decode().strip()
env = os.environ.copy()
print("GIT_COMMIT_HASH", commit_hash)
env["GIT_COMMIT_HASH"] = commit_hash


Path("measurements").mkdir(exist_ok=True)

date = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"measurements/output_{input}_seq_{date}.json"

subprocess.run(["make", "cliftinstr"], check=True)


with open(output_path, "w") as f:
    subprocess.run(["./cliftinstr", "-m", "llama3.2_1b.bin", "-n", "20", "-p", "The son applies for a credit card with an identity already stolen. He takes the card and spends almost all"], stdout=f, check=True,env=env)

file = output_path

GIT_COMMIT_HASH f9cddb23c69bf53a9c28409ecf75ae69ca48f7a1
gcc -DINSTR -O3 -ffast-math -fno-omit-frame-pointer -Wall -o cliftinstr clift.c -lm


clift.c: In function ‘generate’:
clift.c:1398:11: warning: unused variable ‘current’ [-Wunused-variable]
 1398 |       int current = sequence[sequence_len - 1];
      |           ^~~~~~~
In file included from clift.c:3:
clift.c: In function ‘main’:
instrumentor.h:98:3: warning: ‘print_json’ accessing 8 bytes in a region of size 0 [-Wstringop-overflow=]
   98 |   print_json( \
      |   ^~~~~~~~~~~~~
   99 |       string_values_1_add, string_values_1, string_values_2, string_values_3 \
      |       ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  100 |   )
      |   ~
clift.c:1590:3: note: in expansion of macro ‘PRINT_JSON’
 1590 |   PRINT_JSON(string_values_1_add,string_values_1,string_values_2,string_values_3);
      |   ^~~~~~~~~~
instrumentor.h:98:3: note: referencing argument 4 of type ‘char *[0]’
   98 |   print_json( \
      |   ^~~~~~~~~~~~~
   99 |       string_values_1_add, string_values_1, string_values_2, string_values_3 \
      |       ~~~~~~~~~~~

In [4]:
import json

def print_json_schema(data, indent=0):
    prefix = '  ' * indent
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{prefix}{key}: {type(value).__name__}")
            print_json_schema(value, indent + 1)
    elif isinstance(data, list):
        print(f"{prefix}- list of {len(data)} items")
        if data:
            print_json_schema(data[0], indent + 1)
    else:
        # leaf node
        pass

with open(file) as f:
    data = json.load(f)

print_json_schema(data)

commit: str
JSON_1_COMPT: dict
  PROMPT_LEN: float
  TOKEN_GENERATED: float
JSON_1: dict
  ENCODE_TIME: float
  FORWARD_TIME_PROMPT: float
  SAMPLE_TIME: float
  FORWARD_TIME_GENERATION: float
JSON_2: dict
  RMSNORM_INIT: list
    - list of 40 items
  MATMUL_QKV: list
    - list of 40 items
  ROPE: list
    - list of 40 items
  ATTENTION_COMPUTATION: list
    - list of 40 items
  MATMUL_OUTPUT_ATTENTION: list
    - list of 40 items
  FFN_RMSNORM: list
    - list of 40 items
  MATMUL_FFN: list
    - list of 40 items
  SwiGLU: list
    - list of 40 items
  MATMUL_OUTPUT_FFN: list
    - list of 40 items
  FINAL_RMSNORM: list
    - list of 40 items
  MAMTUL_LOGITS: list
    - list of 40 items
JSON_3: dict


In [5]:
import json
import pandas as pd
import plotly.express as px

with open(file) as f:
    data = json.load(f)

# On récupère le sous-dictionnaire JSON_1
json1 = data["JSON_1"]

# Création dynamique du DataFrame
df = pd.DataFrame(list(json1.items()), columns=["phase", "time (ms)"])

# Affichage en bar chart
fig = px.bar(
    df,
    x="phase",
    y="time (ms)",
    title="TIming of different sections",
    width=600
)
fig.show()


In [6]:
import json
import pandas as pd
import plotly.express as px

with open(file) as f:
    data = json.load(f)

json2 = data["JSON_2"]

df = pd.DataFrame({
    "component": list(json2.keys()),
    "time (ms)": [values[0] if isinstance(values, list) and values else None for values in json2.values()]
})

df = df.dropna()

fig = px.bar(
    df,
    x="component",
    y="time (ms)",
    title="Component time for PROMPT PROCESSING",

    width=800,
    text_auto=".2f"
)
fig.show()


In [7]:
df_2 = pd.DataFrame({
    "component": list(json2.keys()),
    "time (ms)": [values[1] if isinstance(values, list) and values else None for values in json2.values()]
})
df = df.dropna()

fig = px.bar(
    df,
    x="component",
    y="time (ms)",
    title="Component time for CACHE LOADING",
    log_y=True,
    width=800,
    text_auto=".2f"
)
fig.show()

In [ ]:
import json
import pandas as pd
import plotly.express as px

with open(file) as f:
    data = json.load(f)

json2 = data["JSON_2"]

df = pd.DataFrame({
    "component": list(json2.keys()),
    "time (ms)": [
        sum(values[2:]) / len(values[2:]) if isinstance(values, list) and len(values) > 2 else None
        for values in json2.values()
    ]
})

df = df.dropna()

fig = px.bar(
    df,
    x="component",
    y="time (ms)",
    title="Average component time for TOKEN GENERATION",

    text_auto='.2f',
    width=800
)
fig.show()


In [9]:
total = sum(values[0] for values in json2.values() if isinstance(values, list) and values)
print(total)

4342.627600000001


In [10]:
import json
import pandas as pd
import plotly.express as px

with open(file) as f:
    data = json.load(f)

json2 = data["JSON_2"]

rows = []
for component, values in json2.items():
    if not isinstance(values, list):
        continue
    cleaned = [v for v in values[2:] if v != 0]
    for i, v in enumerate(cleaned):
        rows.append({
            "step": i,
            "time": v,
            "component": component
        })

df = pd.DataFrame(rows)

fig = px.line(
    df,
    x="step",
    y="time",
    color="component",
    markers=True,
    labels={"time": "time (ms)"},
    title="Per-step time per component during token generation"
)

fig.show()


In [11]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

with open(file) as f:
    data = json.load(f)

forward = data["forward_instr"]
keys = [
    "rmsnorm_first_time",
    "matmul_qkv_time",
    "rope_time",
    "multihead_time",
    "matmul_output_attention_time",
    "FFN_rmsnorm_time",
    "matmul_FFN_time",
    "swiGLU_time",
    "matmul_output_FFN_time",
    "rmsnorm_final_time",
    "matmul_logits_time"
]

rows = []
for k in keys:
    values = [v for v in forward[k] if v != 0]
    rows += [(k, v) for v in values]

df = pd.DataFrame(rows, columns=["phase",  "time (in ms)"])

px.box(df, x="phase", y="time (in ms)",log_y = False,title = "Boxplot of difference phase per execution")


KeyError: 'forward_instr'

In [ ]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

with open(file) as f:
    data = json.load(f)

json2 = data["JSON_2"]

first_values = {
    k: v[0] for k, v in json2.items()
    if isinstance(v, list) and len(v) > 0
}
total_first = sum(first_values.values())
percent_first = {
    k: v / total_first * 100 for k, v in first_values.items()
}

df_first = pd.DataFrame({
    "component": list(percent_first.keys()),
    "percentage": list(percent_first.values())
}).sort_values("percentage", ascending=False)

fig1 = px.bar(
    df_first,
    x="component",
    y="percentage",
    title="Percentage of time for PROMPT_PROCESSING",
    text_auto=".2f"
)
fig1.show()

main_values = {
    k: sum(v[2:]) for k, v in json2.items()
    if isinstance(v, list) and len(v) > 2
}
total_main = sum(main_values.values())
percent_main = {
    k: v / total_main * 100 for k, v in main_values.items()
}

df_main = pd.DataFrame({
    "component": list(percent_main.keys()),
    "percentage": list(percent_main.values())
}).sort_values("percentage", ascending=False)

fig2 = px.bar(
    df_main,
    x="component",
    y="percentage",
    title="Percentage of time for TOKEN_GENERATION",
    text_auto=".2f"
)
fig2.show()


# 1. Multi thread comparison

## 2. ROPE Comparaison

In [ ]:
file1 = "measurements/output_llama_3.bin_seq_20250617_165939.json"
file2 = "measurements/output_llama_3.bin_seq_20250618_174751.json"

In [ ]:
import json
import pandas as pd
import numpy as np
import plotly.express as px


def extract_metrics(path):
    with open(path) as f:
        data = json.load(f)
        rope = data["JSON_2"]["ROPE"]
        return {"PROMPT_PROCESSING": rope[0], "TOKEN_GENERATION": np.mean(rope[2:])}


metrics1 = extract_metrics(file1)
metrics2 = extract_metrics(file2)

speedups = {"stage": [], "speedup (%)": [], "delta" : []}

for key in ["PROMPT_PROCESSING", "TOKEN_GENERATION"]:
    ratio = 100 * metrics1[key] / metrics2[key]
    speedups["stage"].append(key)
    speedups["speedup (%)"].append(ratio)
    speedups["delta"].append(metrics2[key] - metrics1[key])

df = pd.DataFrame(speedups)

fig = px.bar(df, x="stage", y="speedup (%)", title="ROPE speedup with pre-computed Rope", width=500)
fig.show()

In [ ]:
df

,stage,speedup (%),delta
0,PROMPT_PROCESSING,749.676106,-3.510200
1,TOKEN_GENERATION,280.978270,-0.064743
